In [ ]:
#| default_exp benchmarking.benchmark
#| export

# Constants for benchmarking defaults
DEFAULT_WARMUP_RUNS = 5  # Default warmup runs for JIT compilation and cache warming
DEFAULT_MEASUREMENT_RUNS = 10  # Default measurement runs for statistical significance

# Module 19: Benchmarking - TorchPerf Olympics Preparation

**IMPORTANT - hasattr() Usage in This Module:**
This module uses hasattr() throughout for duck-typing and polymorphic benchmarking.
This is LEGITIMATE because:
1. Benchmarking framework must work with ANY model type (PyTorch, TinyTorch, custom)
2. Different frameworks use different method names (forward vs predict vs __call__)
3. We need runtime introspection for maximum compatibility
4. This is the CORRECT use of hasattr() for framework-agnostic tooling

Welcome to the final implementation module! You've learned individual optimization techniques in Modules 14-18. Now you'll build the benchmarking infrastructure that powers **TorchPerf Olympics** - the capstone competition framework.

## 🔗 Prerequisites & Progress
**You've Built**: Complete ML framework with profiling, acceleration, quantization, and compression
**You'll Build**: TorchPerf benchmarking system for fair model comparison and capstone submission
**You'll Enable**: Systematic optimization combination and competitive performance evaluation

**Connection Map**:
```
Individual Optimizations (M14-18) → Benchmarking (M19) → TorchPerf Olympics (Capstone)
(techniques)                        (evaluation)         (competition)
```

## 🏅 TorchPerf Olympics: The Capstone Framework

The TorchPerf Olympics is your capstone competition! Choose your event:
- 🏃 **Latency Sprint**: Minimize inference time (fastest model wins)
- 🏋️ **Memory Challenge**: Minimize model size (smallest footprint wins)  
- 🎯 **Accuracy Contest**: Maximize accuracy within constraints
- 🏋️‍♂️ **All-Around**: Best balanced performance across all metrics
- 🚀 **Extreme Push**: Most aggressive optimization while staying viable

## Learning Objectives
By the end of this module, you will:
1. Implement professional benchmarking infrastructure with statistical rigor
2. Learn to combine optimization techniques strategically (order matters!)
3. Build the TorchPerf class - your standardized capstone submission framework
4. Understand ablation studies and systematic performance evaluation

🔥 Carry the torch. Optimize the model. Win the gold! 🏅

## 📦 Where This Code Lives in the Final Package

**Learning Side:** You work in `modules/19_benchmarking/benchmarking_dev.py`
**Building Side:** Code exports to `tinytorch.benchmarking.benchmark`

**How to use this module (after running `tito module complete 19`):**

```python
from tinytorch.benchmarking.benchmark import Benchmark, OlympicEvent

# For capstone submission:
benchmark = Benchmark([baseline_model, optimized_model],
                     [{"name": "baseline"}, {"name": "optimized"}])
results = benchmark.run_latency_benchmark()
```

**Why this matters:**
- **Learning:** Complete benchmarking ecosystem in one focused module for rigorous evaluation
- **TorchPerf Olympics:** The Benchmark class provides the standardized framework for capstone submissions
- **Consistency:** All benchmarking operations and reporting in benchmarking.benchmark
- **Integration:** Works seamlessly with optimization modules (M14-18) for complete systems evaluation

# 1. Introduction - What is Fair Benchmarking?

Benchmarking in ML systems isn't just timing code - it's about making fair, reproducible comparisons that guide real optimization decisions. Think of it like standardized testing: everyone takes the same test under the same conditions.

Consider comparing three models: a base CNN, a quantized version, and a pruned version. Without proper benchmarking, you might conclude the quantized model is "fastest" because you measured it when your CPU was idle, while testing the others during peak system load. Fair benchmarking controls for these variables.

The challenge: ML models have multiple competing objectives (accuracy vs speed vs memory), measurements can be noisy, and "faster" depends on your hardware and use case.

## Benchmarking as a Systems Engineering Discipline

Professional ML benchmarking requires understanding measurement uncertainty and controlling for confounding factors:

**Statistical Foundations**: We need enough measurements to achieve statistical significance. Running a model once tells you nothing about its true performance - you need distributions.

**System Noise Sources**:
- **Thermal throttling**: CPU frequency drops when hot
- **Background processes**: OS interrupts and other applications
- **Memory pressure**: Garbage collection, cache misses
- **Network interference**: For distributed models

**Fair Comparison Requirements**:
- Same hardware configuration
- Same input data distributions
- Same measurement methodology
- Statistical significance testing

This module builds infrastructure that addresses all these challenges while generating actionable insights for optimization decisions.

# 2. Mathematical Foundations - Statistics for Performance Engineering

Benchmarking is applied statistics. We measure noisy processes (model inference) and need to extract reliable insights about their true performance characteristics.

## Central Limit Theorem in Practice

When you run a model many times, the distribution of measurements approaches normal (regardless of the underlying noise distribution). This lets us:
- Compute confidence intervals for the true mean
- Detect statistically significant differences between models
- Control for measurement variance

```
Single measurement: Meaningless
Few measurements: Unreliable
Many measurements: Statistical confidence
```

## Multi-Objective Optimization Theory

ML systems exist on a **Pareto frontier** - you can't simultaneously maximize accuracy and minimize latency without trade-offs. Good benchmarks reveal this frontier:

```
Accuracy
    ↑
    |  A ●     ← Model A: High accuracy, high latency
    |
    |    B ●  ← Model B: Balanced trade-off
    |
    |      C ●← Model C: Low accuracy, low latency
    |__________→ Latency (lower is better)
```

The goal: Find the optimal operating point for your specific constraints.

## Measurement Uncertainty and Error Propagation

Every measurement has uncertainty. When combining metrics (like accuracy per joule), uncertainties compound:

- **Systematic errors**: Consistent bias (timer overhead, warmup effects)
- **Random errors**: Statistical noise (thermal variation, OS scheduling)
- **Propagated errors**: How uncertainty spreads through calculations

Professional benchmarking quantifies and minimizes these uncertainties.

In [ ]:
import sys, os
sys.path.append(os.path.abspath(".."))

import numpy as np
import time
import statistics
import os
import tracemalloc
from typing import Dict, List, Tuple, Any, Optional, Callable, Union
from dataclasses import dataclass, field
from pathlib import Path
import json
import platform
from contextlib import contextmanager
import warnings

# Optional dependency for visualization only
try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_AVAILABLE = True
except ImportError:
    MATPLOTLIB_AVAILABLE = False
    # Create minimal fallback for when matplotlib is not available
    class plt:
        @staticmethod
        def subplots(*args, **kwargs):
            return None, None
        @staticmethod
        def figure(*args, **kwargs):
            return None
        @staticmethod
        def scatter(*args, **kwargs):
            pass
        @staticmethod
        def annotate(*args, **kwargs):
            pass
        @staticmethod
        def xlabel(*args, **kwargs):
            pass
        @staticmethod
        def ylabel(*args, **kwargs):
            pass
        @staticmethod
        def title(*args, **kwargs):
            pass
        @staticmethod
        def grid(*args, **kwargs):
            pass
        @staticmethod
        def tight_layout(*args, **kwargs):
            pass
        @staticmethod
        def savefig(*args, **kwargs):
            pass
        @staticmethod
        def show(*args, **kwargs):
            pass

# Import Profiler from Module 14 for measurement reuse
from core.profiler import Profiler

In [ ]:
from enum import Enum

#| export
class OlympicEvent(Enum):
    """
    TorchPerf Olympics event categories.
    
    Each event optimizes for different objectives with specific constraints.
    Students choose their event and compete for medals!
    """
    LATENCY_SPRINT = "latency_sprint"      # Minimize latency (accuracy >= 85%)
    MEMORY_CHALLENGE = "memory_challenge"   # Minimize memory (accuracy >= 85%)
    ACCURACY_CONTEST = "accuracy_contest"   # Maximize accuracy (latency < 100ms, memory < 10MB)
    ALL_AROUND = "all_around"               # Best balanced score across all metrics
    EXTREME_PUSH = "extreme_push"           # Most aggressive optimization (accuracy >= 80%)

# 3. Implementation - Building Professional Benchmarking Infrastructure

We'll build a comprehensive benchmarking system that handles statistical analysis, multi-dimensional comparison, and automated reporting. Each component builds toward production-quality evaluation tools.

The architecture follows a hierarchical design:
```
Profiler (Module 14) ← Base measurement tools
       ↓
BenchmarkResult ← Statistical container for measurements
       ↓
Benchmark ← Uses Profiler + adds multi-model comparison
       ↓
BenchmarkSuite ← Multi-metric comprehensive evaluation
       ↓
TinyMLPerf ← Standardized industry-style benchmarks
```

**Key Architectural Decision**: The `Benchmark` class reuses `Profiler` from Module 14 for individual model measurements, then adds statistical comparison across multiple models. This demonstrates proper systems architecture - build once, reuse everywhere!

Each level adds capability while maintaining statistical rigor at the foundation.

## BenchmarkResult - Statistical Analysis Container

Before measuring anything, we need a robust container that stores measurements and computes statistical properties. This is the foundation of all our benchmarking.

### Why Statistical Analysis Matters

Single measurements are meaningless in performance engineering. Consider timing a model:
- Run 1: 1.2ms (CPU was idle)
- Run 2: 3.1ms (background process started)
- Run 3: 1.4ms (CPU returned to normal)

Without statistics, which number do you trust? BenchmarkResult solves this by:
- Computing confidence intervals for the true mean
- Detecting outliers and measurement noise
- Providing uncertainty estimates for decision making

### Statistical Properties We Track

```
Raw measurements: [1.2, 3.1, 1.4, 1.3, 1.5, 1.1, 1.6]
                           ↓
        Statistical Analysis
                           ↓
Mean: 1.46ms ± 0.25ms (95% confidence interval)
Median: 1.4ms (less sensitive to outliers)
CV: 17% (coefficient of variation - relative noise)
```

The confidence interval tells us: "We're 95% confident the true mean latency is between 1.21ms and 1.71ms." This guides optimization decisions with statistical backing.

In [ ]:
#| export

@dataclass
class BenchmarkResult:
    """
    Container for benchmark measurements with statistical analysis.

    TODO: Implement a robust result container that stores measurements and metadata

    APPROACH:
    1. Store raw measurements and computed statistics
    2. Include metadata about test conditions
    3. Provide methods for statistical analysis
    4. Support serialization for result persistence

    EXAMPLE:
    >>> result = BenchmarkResult("model_accuracy", [0.95, 0.94, 0.96])
    >>> print(f"Mean: {result.mean:.3f} ± {result.std:.3f}")
    Mean: 0.950 ± 0.010

    HINTS:
    - Use statistics module for robust mean/std calculations
    - Store both raw data and summary statistics
    - Include confidence intervals for professional reporting
    """
    ### BEGIN SOLUTION

    ### END SOLUTION

def test_unit_benchmark_result():
    """🔬 Test BenchmarkResult statistical calculations."""
    print("🔬 Unit Test: BenchmarkResult...")

    # Test basic statistics
    values = [1.0, 2.0, 3.0, 4.0, 5.0]
    result = BenchmarkResult("test_metric", values)

    assert result.mean == 3.0
    assert abs(result.std - statistics.stdev(values)) < 1e-10
    assert result.median == 3.0
    assert result.min_val == 1.0
    assert result.max_val == 5.0
    assert result.count == 5

    # Test confidence intervals
    assert result.ci_lower < result.mean < result.ci_upper

    # Test serialization
    result_dict = result.to_dict()
    assert result_dict['metric_name'] == "test_metric"
    assert result_dict['mean'] == 3.0

    print("✅ BenchmarkResult works correctly!")

if __name__ == "__main__":
    test_unit_benchmark_result()

## High-Precision Timing Infrastructure

Accurate timing is the foundation of performance benchmarking. System clocks have different precision and behavior, so we need a robust timing mechanism.

### Timing Challenges in Practice

Consider what happens when you time a function:
```
User calls: time.time()
            ↓
Operating System scheduling delays (μs to ms)
            ↓
Timer system call overhead (~1μs)
            ↓
Hardware clock resolution (ns to μs)
            ↓
Your measurement
```

For microsecond-precision timing, each of these can introduce significant error.

### Why perf_counter() Matters

Python's `time.perf_counter()` is specifically designed for interval measurement:
- **Monotonic**: Never goes backwards (unaffected by system clock adjustments)
- **High resolution**: Typically nanosecond precision
- **Low overhead**: Optimized system call

### Timing Best Practices

```
Context Manager Pattern:
┌─────────────────┐
│  with timer():  │ ← Start timing
│    operation()  │ ← Your code runs
│  # End timing   │ ← Automatic cleanup
└─────────────────┘
    ↓
elapsed = timer.elapsed
```

This pattern ensures timing starts/stops correctly even if exceptions occur.

In [ ]:
@contextmanager
def precise_timer():
    """
    High-precision timing context manager for benchmarking.

    TODO: Implement a context manager that provides accurate timing measurements

    APPROACH:
    1. Use time.perf_counter() for high precision
    2. Handle potential interruptions and system noise
    3. Return elapsed time when context exits
    4. Provide warmup capability for JIT compilation

    Yields:
        Timer object with .elapsed attribute (set after context exits)

    EXAMPLE:
    >>> with precise_timer() as timer:
    ...     time.sleep(0.1)  # Some operation
    >>> print(f"Elapsed: {timer.elapsed:.4f}s")
    Elapsed: 0.1001s

    HINTS:
    - perf_counter() is monotonic and high-resolution
    - Store start time in __enter__, compute elapsed in __exit__
    - Handle any exceptions gracefully
    """
    ### BEGIN SOLUTION

    ### END SOLUTION

def test_unit_precise_timer():
    """🔬 Test precise_timer context manager."""
    print("🔬 Unit Test: precise_timer...")

    # Test basic timing
    with precise_timer() as timer:
        time.sleep(0.01)  # 10ms sleep

    # Should be close to 0.01 seconds (allow some variance)
    assert 0.005 < timer.elapsed < 0.05, f"Expected ~0.01s, got {timer.elapsed}s"

    # Test multiple uses
    times = []
    for _ in range(3):
        with precise_timer() as timer:
            time.sleep(0.001)  # 1ms sleep
        times.append(timer.elapsed)

    # All times should be reasonably close
    assert all(0.0005 < t < 0.01 for t in times)

    print("✅ precise_timer works correctly!")

if __name__ == "__main__":
    test_unit_precise_timer()

## Benchmark Class - Core Measurement Engine

The Benchmark class implements the core measurement logic for different metrics. It handles the complex orchestration of multiple models, datasets, and measurement protocols.

### Benchmark Architecture Overview

```
Benchmark Execution Flow:
┌─────────────┐    ┌──────────────┐    ┌─────────────────┐
│   Models    │    │   Datasets   │    │ Measurement     │
│ [M1, M2...] │ →  │ [D1, D2...]  │ →  │ Protocol        │
└─────────────┘    └──────────────┘    └─────────────────┘
                                               ↓
                           ┌─────────────────────────────────┐
                           │        Benchmark Loop           │
                           │ 1. Warmup runs (JIT, cache)     │
                           │ 2. Measurement runs (statistics)│
                           │ 3. System info capture          │
                           │ 4. Result aggregation           │
                           └─────────────────────────────────┘
                                        ↓
                        ┌────────────────────────────────────┐
                        │          BenchmarkResult           │
                        │ • Statistical analysis             │
                        │ • Confidence intervals             │
                        │ • Metadata (system, conditions)    │
                        └────────────────────────────────────┘
```

### Why Warmup Runs Matter

Modern systems have multiple layers of adaptation:
- **JIT compilation**: Code gets faster after being run several times
- **CPU frequency scaling**: Processors ramp up under load
- **Cache warming**: Data gets loaded into faster memory
- **Branch prediction**: CPU learns common execution paths

Without warmup, your first few measurements don't represent steady-state performance.

### Multiple Benchmark Types

Different metrics require different measurement strategies:

**Latency Benchmarking**:
- Focus: Time per inference
- Key factors: Input size, model complexity, hardware utilization
- Measurement: High-precision timing of forward pass

**Accuracy Benchmarking**:
- Focus: Quality of predictions
- Key factors: Dataset representativeness, evaluation protocol
- Measurement: Correct predictions / total predictions

**Memory Benchmarking**:
- Focus: Peak and average memory usage
- Key factors: Model size, batch size, intermediate activations
- Measurement: Process memory monitoring during inference

In [ ]:
#| export
class Benchmark:
    """
    Professional benchmarking system for ML models and operations.

    TODO: Implement a comprehensive benchmark runner with statistical rigor

    APPROACH:
    1. Support multiple models, datasets, and metrics
    2. Run repeated measurements with proper warmup
    3. Control for system variance and compute confidence intervals
    4. Generate structured results for analysis

    EXAMPLE:
    >>> benchmark = Benchmark(models=[model1, model2], datasets=[test_data])
    >>> results = benchmark.run_accuracy_benchmark()
    >>> benchmark.plot_results(results)

    HINTS:
    - Use warmup runs to stabilize performance
    - Collect multiple samples for statistical significance
    - Store metadata about system conditions
    - Provide different benchmark types (accuracy, latency, memory)
    """
    ### BEGIN SOLUTION

    ### END SOLUTION

def test_unit_benchmark():
    """🔬 Test Benchmark class functionality."""
    print("🔬 Unit Test: Benchmark...")

    # Create mock models for testing
    class MockModel:
        def __init__(self, name):
            self.name = name

        def forward(self, x):
            time.sleep(0.001)  # Simulate computation
            return x

    models = [MockModel("fast_model"), MockModel("slow_model")]
    datasets = [{"data": "test1"}, {"data": "test2"}]

    benchmark = Benchmark(models, datasets, warmup_runs=2, measurement_runs=3)

    # Test latency benchmark
    latency_results = benchmark.run_latency_benchmark()
    assert len(latency_results) == 2
    assert "fast_model" in latency_results
    assert all(isinstance(result, BenchmarkResult) for result in latency_results.values())

    # Test accuracy benchmark
    accuracy_results = benchmark.run_accuracy_benchmark()
    assert len(accuracy_results) == 2
    assert all(0 <= result.mean <= 1 for result in accuracy_results.values())

    # Test memory benchmark
    memory_results = benchmark.run_memory_benchmark()
    assert len(memory_results) == 2
    assert all(result.mean >= 0 for result in memory_results.values())

    # Test comparison (returns list of dicts, not DataFrame)
    comparison_data = benchmark.compare_models("latency")
    assert len(comparison_data) == 2
    assert isinstance(comparison_data, list)
    assert all(isinstance(item, dict) for item in comparison_data)
    assert "model" in comparison_data[0]
    assert "mean" in comparison_data[0]

    print("✅ Benchmark works correctly!")

if __name__ == "__main__":
    test_unit_benchmark()

## BenchmarkSuite - Comprehensive Multi-Metric Evaluation

The BenchmarkSuite orchestrates multiple benchmark types and generates comprehensive reports. This is where individual measurements become actionable engineering insights.

### Why Multi-Metric Analysis Matters

Single metrics mislead. Consider these three models:
- **Model A**: 95% accuracy, 100ms latency, 50MB memory
- **Model B**: 90% accuracy, 20ms latency, 10MB memory
- **Model C**: 85% accuracy, 10ms latency, 5MB memory

Which is "best"? It depends on your constraints:
- **Server deployment**: Model A (accuracy matters most)
- **Mobile app**: Model C (memory/latency critical)
- **Edge device**: Model B (balanced trade-off)

### Multi-Dimensional Comparison Workflow

```
BenchmarkSuite Execution Pipeline:
┌──────────────┐
│   Models     │ ← Input: List of models to compare
│ [M1,M2,M3]   │
└──────┬───────┘
       ↓
┌──────────────┐
│ Metric Types │ ← Run each benchmark type
│ • Latency    │
│ • Accuracy   │
│ • Memory     │
│ • Energy     │
└──────┬───────┘
       ↓
┌──────────────┐
│ Result       │ ← Aggregate into unified view
│ Aggregation  │
└──────┬───────┘
       ↓
┌──────────────┐
│ Analysis &   │ ← Generate insights
│ Reporting    │   • Best performer per metric
│              │   • Trade-off analysis
│              │   • Use case recommendations
└──────────────┘
```

### Pareto Frontier Analysis

The suite automatically identifies Pareto-optimal solutions - models that aren't strictly dominated by others across all metrics. This reveals the true trade-off space for optimization decisions.

### Energy Efficiency Modeling

Since direct energy measurement requires specialized hardware, we estimate energy based on computational complexity and memory usage. This provides actionable insights for battery-powered deployments.

In [ ]:
#| export
class BenchmarkSuite:
    """
    Comprehensive benchmark suite for ML systems evaluation.

    TODO: Implement a full benchmark suite that runs multiple test categories

    APPROACH:
    1. Combine multiple benchmark types (latency, accuracy, memory, energy)
    2. Generate comprehensive reports with visualizations
    3. Support different model categories and hardware configurations
    4. Provide recommendations based on results

    EXAMPLE:
    >>> suite = BenchmarkSuite(models, datasets)
    >>> report = suite.run_full_benchmark()
    >>> suite.generate_report(report)

    HINTS:
    - Organize results by benchmark type and model
    - Create Pareto frontier analysis for trade-offs
    - Include system information and test conditions
    - Generate actionable insights and recommendations
    """
    ### BEGIN SOLUTION

    ### END SOLUTION

def test_unit_benchmark_suite():
    """🔬 Test BenchmarkSuite comprehensive functionality."""
    print("🔬 Unit Test: BenchmarkSuite...")

    # Create mock models
    class MockModel:
        def __init__(self, name):
            self.name = name

        def forward(self, x):
            time.sleep(0.001)
            return x

    models = [MockModel("efficient_model"), MockModel("accurate_model")]
    datasets = [{"test": "data"}]

    # Create temporary directory for test output
    import tempfile
    with tempfile.TemporaryDirectory() as tmp_dir:
        suite = BenchmarkSuite(models, datasets, output_dir=tmp_dir)

        # Run full benchmark
        results = suite.run_full_benchmark()

        # Verify all benchmark types completed
        assert 'latency' in results
        assert 'accuracy' in results
        assert 'memory' in results
        assert 'energy' in results

        # Verify results structure
        for metric_results in results.values():
            assert len(metric_results) == 2  # Two models
            assert all(isinstance(result, BenchmarkResult) for result in metric_results.values())

        # Test report generation
        report = suite.generate_report()
        assert "Benchmark Report" in report
        assert "System Information" in report
        assert "Recommendations" in report

        # Verify files are created
        output_path = Path(tmp_dir)
        assert (output_path / 'benchmark_report.md').exists()

    print("✅ BenchmarkSuite works correctly!")

if __name__ == "__main__":
    test_unit_benchmark_suite()

## TinyMLPerf - Standardized Industry Benchmarking

TinyMLPerf provides standardized benchmarks that enable fair comparison across different systems, similar to how MLPerf works for larger models. This is crucial for reproducible research and industry adoption.

### Why Standardization Matters

Without standards, every team benchmarks differently:
- Different datasets, input sizes, measurement protocols
- Different accuracy metrics, latency definitions
- Different hardware configurations, software stacks

This makes it impossible to compare results across papers, products, or research groups.

### TinyMLPerf Benchmark Architecture

```
TinyMLPerf Benchmark Structure:
┌─────────────────────────────────────────────────────────┐
│                  Benchmark Definition                   │
│ • Standard datasets (CIFAR-10, Speech Commands, etc.)   │
│ • Fixed input shapes and data types                     │
│ • Target accuracy and latency thresholds                │
│ • Measurement protocol (warmup, runs, etc.)             │
└─────────────────────────────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────┐
│                 Execution Protocol                      │
│ 1. Model registration and validation                    │
│ 2. Warmup phase (deterministic random inputs)           │
│ 3. Measurement phase (statistical sampling)             │
│ 4. Accuracy evaluation (ground truth comparison)        │
│ 5. Compliance checking (thresholds, statistical tests)  │
└─────────────────────────────────────────────────────────┘
                           ↓
┌─────────────────────────────────────────────────────────┐
│              Compliance Determination                   │
│ PASS: accuracy ≥ target AND latency ≤ target            │
│ FAIL: Either constraint violated                        │
│ Report: Detailed metrics + system information           │
└─────────────────────────────────────────────────────────┘
```

### Standard Benchmark Tasks

**Keyword Spotting**: Wake word detection from audio
- Input: 1-second 16kHz audio samples
- Task: Binary classification (keyword present/absent)
- Target: 90% accuracy, <100ms latency

**Visual Wake Words**: Person detection in images
- Input: 96×96 RGB images
- Task: Binary classification (person present/absent)
- Target: 80% accuracy, <200ms latency

**Anomaly Detection**: Industrial sensor monitoring
- Input: 640-element sensor feature vectors
- Task: Binary classification (anomaly/normal)
- Target: 85% accuracy, <50ms latency

### Reproducibility Requirements

All TinyMLPerf benchmarks use:
- **Fixed random seeds**: Deterministic input generation
- **Standardized hardware**: Reference implementations for comparison
- **Statistical validation**: Multiple runs with confidence intervals
- **Compliance reporting**: Machine-readable results format

In [ ]:
#| export
class TinyMLPerf:
    """
    TinyMLPerf-style standardized benchmarking for edge ML systems.

    TODO: Implement standardized benchmarks following TinyMLPerf methodology

    APPROACH:
    1. Define standard benchmark tasks and datasets
    2. Implement standardized measurement protocols
    3. Ensure reproducible results across different systems
    4. Generate compliance reports for fair comparison

    EXAMPLE:
    >>> perf = TinyMLPerf()
    >>> results = perf.run_keyword_spotting_benchmark(model)
    >>> perf.generate_compliance_report(results)

    HINTS:
    - Use fixed random seeds for reproducibility
    - Implement warm-up and measurement phases
    - Follow TinyMLPerf power and latency measurement standards
    - Generate standardized result formats
    """
    ### BEGIN SOLUTION

    ### END SOLUTION

def test_unit_tinymlperf():
    """🔬 Test TinyMLPerf standardized benchmarking."""
    print("🔬 Unit Test: TinyMLPerf...")

    # Create mock model for testing
    class MockModel:
        def __init__(self, name):
            self.name = name

        def forward(self, x):
            time.sleep(0.001)  # Simulate computation
            # Return appropriate output shape for different benchmarks
            if hasattr(x, 'shape'):
                if len(x.shape) == 2:  # Audio/sequence
                    return np.random.rand(2)  # Binary classification
                else:  # Image
                    return np.random.rand(10)  # Multi-class
            return np.random.rand(2)

    model = MockModel("test_model")
    perf = TinyMLPerf(random_seed=42)

    # Test individual benchmark
    result = perf.run_standard_benchmark(model, 'keyword_spotting', num_runs=5)

    # Verify result structure
    required_keys = ['accuracy', 'mean_latency_ms', 'throughput_fps', 'compliant']
    assert all(key in result for key in required_keys)
    assert 0 <= result['accuracy'] <= 1
    assert result['mean_latency_ms'] > 0
    assert result['throughput_fps'] > 0

    # Test full benchmark suite (with fewer runs for speed)
    import tempfile
    with tempfile.TemporaryDirectory() as tmp_dir:
        # Run subset of benchmarks for testing
        subset_results = {}
        for benchmark in ['keyword_spotting', 'image_classification']:
            subset_results[benchmark] = perf.run_standard_benchmark(model, benchmark, num_runs=3)

        # Test compliance report generation
        report_path = f"{tmp_dir}/test_report.json"
        summary = perf.generate_compliance_report(subset_results, report_path)

        # Verify report was created
        assert Path(report_path).exists()
        assert "TinyMLPerf Compliance Report" in summary
        assert "Compliance Rate" in summary

    print("✅ TinyMLPerf works correctly!")

if __name__ == "__main__":
    test_unit_tinymlperf()

# 4. Integration - Building Complete Benchmark Workflows

Now we'll integrate all our benchmarking components into complete workflows that demonstrate professional ML systems evaluation. This integration shows how to combine statistical rigor with practical insights.

The integration layer connects individual measurements into actionable engineering insights. This is where benchmarking becomes a decision-making tool rather than just data collection.

## Workflow Architecture

```
Integration Workflow Pipeline:
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│ Model Variants  │    │ Optimization    │    │ Use Case        │
│ • Base model    │ →  │ Techniques      │ →  │ Analysis        │
│ • Quantized     │    │ • Accuracy loss │    │ • Mobile        │
│ • Pruned        │    │ • Speed gain    │    │ • Server        │
│ • Distilled     │    │ • Memory save   │    │ • Edge          │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

This workflow helps answer questions like:
- "Which optimization gives the best accuracy/latency trade-off?"
- "What's the memory budget impact of each technique?"
- "Which model should I deploy for mobile vs server?"

## Optimization Comparison Engine

Before implementing the comparison function, let's understand what makes optimization comparison challenging and valuable.

### Why Optimization Comparison is Complex

When you optimize a model, you're making trade-offs across multiple dimensions simultaneously:

```
Optimization Impact Matrix:
                   Accuracy    Latency    Memory    Energy
Quantization        -5%        +2.1x      +2.0x     +1.8x
Pruning            -2%        +1.4x      +3.2x     +1.3x
Knowledge Distill. -8%        +1.9x      +1.5x     +1.7x
```

The challenge: Which is "best"? It depends entirely on your deployment constraints.

### Multi-Objective Decision Framework

Our comparison engine implements a decision framework that:

1. **Measures all dimensions**: Don't optimize in isolation
2. **Calculates efficiency ratios**: Accuracy per MB, accuracy per ms
3. **Identifies Pareto frontiers**: Models that aren't dominated in all metrics
4. **Generates use-case recommendations**: Tailored to specific constraints

### Recommendation Algorithm

```
For each use case:
├── Latency-critical (real-time apps)
│   └── Optimize: min(latency) subject to accuracy > threshold
├── Memory-constrained (mobile/IoT)
│   └── Optimize: min(memory) subject to accuracy > threshold
├── Accuracy-preservation (quality-critical)
│   └── Optimize: max(accuracy) subject to latency < threshold
└── Balanced (general deployment)
    └── Optimize: weighted combination of all factors
```

This principled approach ensures recommendations match real deployment needs.

In [ ]:
def compare_optimization_techniques(base_model: Any, optimized_models: List[Any],
                                  datasets: List[Any]) -> Dict[str, Any]:
    """
    Compare base model against various optimization techniques.

    TODO: Implement comprehensive comparison of optimization approaches

    APPROACH:
    1. Run benchmarks on base model and all optimized variants
    2. Calculate improvement ratios and trade-offs
    3. Generate insights about which optimizations work best
    4. Create recommendation matrix for different use cases

    Args:
        base_model: Baseline model (unoptimized)
        optimized_models: List of models with different optimizations applied
        datasets: List of datasets for evaluation

    Returns:
        Dictionary with 'base_metrics', 'optimized_results', 'improvements', 'recommendations'

    EXAMPLE:
    >>> models = [base_model, quantized_model, pruned_model, distilled_model]
    >>> results = compare_optimization_techniques(base_model, models[1:], datasets)
    >>> print(results['recommendations'])

    HINTS:
    - Compare accuracy retention vs speed/memory improvements
    - Calculate efficiency metrics (accuracy per MB, accuracy per ms)
    - Identify Pareto-optimal solutions
    - Generate actionable recommendations for different scenarios
    """
    ### BEGIN SOLUTION

    ### END SOLUTION

def test_unit_optimization_comparison():
    """🔬 Test optimization comparison functionality."""
    print("🔬 Unit Test: compare_optimization_techniques...")

    # Create mock models with different characteristics
    class MockModel:
        def __init__(self, name, latency_factor=1.0, accuracy_factor=1.0, memory_factor=1.0):
            self.name = name
            self.latency_factor = latency_factor
            self.accuracy_factor = accuracy_factor
            self.memory_factor = memory_factor

        def forward(self, x):
            time.sleep(0.001 * self.latency_factor)
            return x

    # Base model and optimized variants
    base_model = MockModel("base_model", latency_factor=1.0, accuracy_factor=1.0, memory_factor=1.0)
    quantized_model = MockModel("quantized_model", latency_factor=0.7, accuracy_factor=0.95, memory_factor=0.5)
    pruned_model = MockModel("pruned_model", latency_factor=0.8, accuracy_factor=0.98, memory_factor=0.3)

    datasets = [{"test": "data"}]

    # Run comparison
    results = compare_optimization_techniques(base_model, [quantized_model, pruned_model], datasets)

    # Verify results structure
    assert 'base_model' in results
    assert 'optimized_results' in results
    assert 'improvements' in results
    assert 'recommendations' in results

    # Verify improvements were calculated
    assert len(results['improvements']) == 2  # Two optimized models

    # Verify recommendations were generated
    recommendations = results['recommendations']
    assert 'for_latency_critical' in recommendations
    assert 'for_memory_constrained' in recommendations
    assert 'for_accuracy_preservation' in recommendations
    assert 'for_balanced_deployment' in recommendations

    print("✅ compare_optimization_techniques works correctly!")

if __name__ == "__main__":
    test_unit_optimization_comparison()

## 4.4 Systems Analysis - Benchmark Variance and Optimization Trade-offs

Understanding measurement variance and optimization trade-offs through systematic analysis.

In [ ]:
def analyze_benchmark_variance():
    """📊 Analyze measurement variance and confidence intervals."""
    print("📊 Analyzing Benchmark Variance")
    print("=" * 60)

    # Simulate benchmarking with different sample sizes
    sample_sizes = [5, 10, 20, 50, 100]
    true_latency = 10.0  # True mean latency in ms
    noise_std = 1.5  # Standard deviation of measurement noise

    print("Effect of Sample Size on Confidence Interval Width:\n")
    print(f"{'Samples':<10} {'Mean (ms)':<15} {'CI Width (ms)':<15} {'Relative Error':<15}")
    print("-" * 60)

    for n_samples in sample_sizes:
        # Simulate measurements
        measurements = np.random.normal(true_latency, noise_std, n_samples)
        mean_latency = np.mean(measurements)
        std_latency = np.std(measurements)

        # Calculate 95% confidence interval
        t_score = 1.96
        margin_error = t_score * (std_latency / np.sqrt(n_samples))
        ci_width = 2 * margin_error
        relative_error = ci_width / mean_latency * 100

        print(f"{n_samples:<10} {mean_latency:<15.2f} {ci_width:<15.2f} {relative_error:<15.1f}%")

    print("\n💡 Key Insights:")
    print("   • More samples reduce confidence interval width")
    print("   • CI width decreases with √n (diminishing returns)")
    print("   • 20-50 samples typically sufficient for <10% error")
    print("   • Statistical rigor requires measuring variance, not just mean")

if __name__ == "__main__":
    analyze_benchmark_variance()

In [ ]:
def analyze_optimization_tradeoffs():
    """📊 Analyze trade-offs between different optimization techniques."""
    print("\n📊 Analyzing Optimization Trade-offs")
    print("=" * 60)

    # Simulated optimization results
    optimizations = {
        'Baseline': {'accuracy': 0.89, 'latency_ms': 45, 'memory_mb': 12, 'energy_j': 2.0},
        'Quantization (INT8)': {'accuracy': 0.88, 'latency_ms': 30, 'memory_mb': 3, 'energy_j': 1.3},
        'Pruning (70%)': {'accuracy': 0.87, 'latency_ms': 35, 'memory_mb': 4, 'energy_j': 1.5},
        'Both (INT8 + 70%)': {'accuracy': 0.85, 'latency_ms': 22, 'memory_mb': 1, 'energy_j': 0.9},
    }

    # Calculate efficiency metrics
    print("\nEfficiency Metrics (higher is better):\n")
    print(f"{'Technique':<25} {'Acc/MB':<12} {'Acc/ms':<12} {'Acc/J':<12}")
    print("-" * 60)

    baseline = optimizations['Baseline']

    for name, metrics in optimizations.items():
        acc_per_mb = metrics['accuracy'] / metrics['memory_mb']
        acc_per_ms = metrics['accuracy'] / metrics['latency_ms']
        acc_per_j = metrics['accuracy'] / metrics['energy_j']

        print(f"{name:<25} {acc_per_mb:<12.3f} {acc_per_ms:<12.4f} {acc_per_j:<12.3f}")

    print("\nPareto Frontier Analysis:")
    print("   • Quantization: Best memory efficiency (0.293 acc/MB)")
    print("   • Pruning: Balanced trade-off")
    print("   • Combined: Maximum resource efficiency, highest accuracy loss")

    print("\n💡 Key Insights:")
    print("   • No single optimization dominates all metrics")
    print("   • Combined optimizations compound benefits and risks")
    print("   • Choose based on deployment constraints (memory vs speed vs accuracy)")
    print("   • Pareto frontier reveals non-dominated solutions")

if __name__ == "__main__":
    analyze_optimization_tradeoffs()

## 4.4 MLPerf Principles - Industry-Standard Benchmarking

MLPerf (created by MLCommons) is the industry-standard ML benchmarking framework. Understanding these principles grounds your capstone competition in professional methodology.

### Core Principles

**Reproducibility:** Fixed hardware specs, software versions, random seeds, and multiple runs for statistical validity.

**Standardization:** Fixed models and datasets enable fair comparison. MLPerf has two divisions:
- **Closed:** Same models/datasets, optimize systems (hardware/software)
- **Open:** Modify models/algorithms, show innovation

**TinyMLPerf:** Edge device benchmarks (<1MB models, <100ms latency, <10mW power) that inspire the capstone.

### Key Takeaways

1. Document everything for reproducibility
2. Use same baseline for fair comparison
3. Measure multiple metrics (accuracy, latency, memory, energy)
4. Optimize for real deployment constraints

**Module 20 capstone** follows TinyMLPerf-style principles!

## 4.5 Combination Strategies - Preparing for TorchPerf Olympics

Strategic optimization combines multiple techniques for different competition objectives. The order matters: quantize-then-prune may preserve accuracy better, while prune-then-quantize may be faster.

### Ablation Studies

Professional ML engineers use ablation studies to understand each optimization's contribution:

```
Baseline:           Accuracy: 89%, Latency: 45ms, Memory: 12MB
+ Quantization:     Accuracy: 88%, Latency: 30ms, Memory: 3MB   (Δ: -1%, -33%, -75%)
+ Pruning:          Accuracy: 87%, Latency: 22ms, Memory: 2MB   (Δ: -1%, -27%, -33%)
+ Kernel Fusion:    Accuracy: 87%, Latency: 18ms, Memory: 2MB   (Δ: 0%, -18%, 0%)
```

### Olympic Event Quick Guide

- **Latency Sprint**: Fusion > Caching > Quantization > Pruning
- **Memory Challenge**: Quantization > Pruning > Compression
- **Accuracy Contest**: High-bit quantization (8-bit), light pruning (30-50%)
- **All-Around**: Balanced INT8 + 60% pruning + selective fusion
- **Extreme Push**: 4-bit quantization + 90% pruning (verify accuracy threshold)

**Key strategy:** Start with one technique, measure impact, add next, repeat!

# 5. Module Integration Test

Final validation that our complete benchmarking system works correctly and integrates properly with all TinyTorch components.

This comprehensive test validates the entire benchmarking ecosystem and ensures it's ready for production use in the final capstone project.

In [ ]:
def test_module():
    """🧪 Module Test: Complete Integration

    Comprehensive test of entire benchmarking module functionality.

    This final test runs before module summary to ensure:
    - All benchmarking components work together correctly
    - Statistical analysis provides reliable results
    - Integration with optimization modules functions properly
    - Professional reporting generates actionable insights
    """
    print("🧪 RUNNING MODULE INTEGRATION TEST")
    print("=" * 50)

    # Run all unit tests
    print("Running unit tests...")
    test_unit_benchmark_result()
    test_unit_precise_timer()
    test_unit_benchmark()
    test_unit_benchmark_suite()
    test_unit_tinymlperf()
    test_unit_optimization_comparison()

    print("\nRunning integration scenarios...")

    # Test realistic benchmarking workflow
    print("🔬 Integration Test: Complete benchmarking workflow...")

    # Create realistic test models
    class RealisticModel:
        def __init__(self, name, characteristics):
            self.name = name
            self.characteristics = characteristics

        def forward(self, x):
            # Simulate different model behaviors
            base_time = self.characteristics.get('base_latency', 0.001)
            variance = self.characteristics.get('variance', 0.0001)
            memory_factor = self.characteristics.get('memory_factor', 1.0)

            # Simulate realistic computation
            time.sleep(max(0, base_time + np.random.normal(0, variance)))

            # Simulate memory usage
            if hasattr(x, 'shape'):
                temp_size = int(np.prod(x.shape) * memory_factor)
                temp_data = np.random.randn(temp_size)
                _ = np.sum(temp_data)  # Use the data

            return x

        def evaluate(self, dataset):
            # Simulate evaluation
            base_acc = self.characteristics.get('base_accuracy', 0.85)
            return base_acc + np.random.normal(0, 0.02)

        def parameters(self):
            # Simulate parameter count - return Tensor objects for compatibility
            from core.tensor import Tensor
            param_count = self.characteristics.get('param_count', 1000000)
            return [Tensor(np.random.randn(param_count))]

    # Create test model suite
    models = [
        RealisticModel("efficient_model", {
            'base_latency': 0.001,
            'base_accuracy': 0.82,
            'memory_factor': 0.5,
            'param_count': 500000
        }),
        RealisticModel("accurate_model", {
            'base_latency': 0.003,
            'base_accuracy': 0.95,
            'memory_factor': 2.0,
            'param_count': 2000000
        }),
        RealisticModel("balanced_model", {
            'base_latency': 0.002,
            'base_accuracy': 0.88,
            'memory_factor': 1.0,
            'param_count': 1000000
        })
    ]

    datasets = [{"test_data": f"dataset_{i}"} for i in range(3)]

    # Test 1: Comprehensive benchmark suite
    print("  Testing comprehensive benchmark suite...")
    suite = BenchmarkSuite(models, datasets)
    results = suite.run_full_benchmark()

    assert 'latency' in results
    assert 'accuracy' in results
    assert 'memory' in results
    assert 'energy' in results

    # Verify all models were tested
    for result_type in results.values():
        assert len(result_type) == len(models)

    # Test 2: Statistical analysis
    print("  Testing statistical analysis...")
    for result_type, model_results in results.items():
        for model_name, result in model_results.items():
            assert isinstance(result, BenchmarkResult)
            assert result.count > 0
            assert result.std >= 0
            assert result.ci_lower <= result.mean <= result.ci_upper

    # Test 3: Report generation
    print("  Testing report generation...")
    report = suite.generate_report()
    assert "Benchmark Report" in report
    assert "System Information" in report
    assert "Recommendations" in report

    # Test 4: TinyMLPerf compliance
    print("  Testing TinyMLPerf compliance...")
    perf = TinyMLPerf(random_seed=42)
    perf_results = perf.run_standard_benchmark(models[0], 'keyword_spotting', num_runs=5)

    required_keys = ['accuracy', 'mean_latency_ms', 'compliant', 'target_accuracy']
    assert all(key in perf_results for key in required_keys)
    assert 0 <= perf_results['accuracy'] <= 1
    assert perf_results['mean_latency_ms'] > 0

    # Test 5: Optimization comparison
    print("  Testing optimization comparison...")
    comparison_results = compare_optimization_techniques(
        models[0], models[1:], datasets[:1]
    )

    assert 'base_model' in comparison_results
    assert 'improvements' in comparison_results
    assert 'recommendations' in comparison_results
    assert len(comparison_results['improvements']) == 2

    # Test 6: Cross-platform compatibility
    print("  Testing cross-platform compatibility...")
    system_info = {
        'platform': platform.platform(),
        'processor': platform.processor(),
        'python_version': platform.python_version()
    }

    # Verify system information is captured
    benchmark = Benchmark(models[:1], datasets[:1])
    assert all(key in benchmark.system_info for key in system_info.keys())

    print("✅ End-to-end benchmarking workflow works!")

    print("\n" + "=" * 50)
    print("🎉 ALL TESTS PASSED! Module ready for export.")
    print("Run: tito module complete 19")

if __name__ == "__main__":
    test_module()

## 🤔 ML Systems Thinking: Benchmarking and Performance Engineering

### Question 1: Statistical Confidence in Measurements
You implemented BenchmarkResult with confidence intervals for measurements.
If you run 20 trials and get mean latency 5.2ms with std dev 0.8ms:
- What's the 95% confidence interval for the true mean? [_____ ms, _____ ms]
- How many more trials would you need to halve the confidence interval width? _____ total trials

### Question 2: Measurement Overhead Analysis
Your precise_timer context manager has microsecond precision, but models run for milliseconds.
For a model that takes 1ms to execute:
- If timer overhead is 10μs, what's the relative error? _____%
- At what model latency does timer overhead become negligible (<1%)? _____ ms

### Question 3: Benchmark Configuration Trade-offs
Your optimize_benchmark_configuration() function tested different warmup/measurement combinations.
For a CI/CD pipeline that runs 100 benchmarks per day:
- Fast config (3s each): _____ minutes total daily
- Accurate config (15s each): _____ minutes total daily
- What's the key trade-off you're making? [accuracy/precision/development velocity]

### Question 4: TinyMLPerf Compliance Metrics
You implemented TinyMLPerf-style standardized benchmarks with target thresholds.
If a model achieves 89% accuracy (target: 90%) and 120ms latency (target: <100ms):
- Is it compliant? [Yes/No] _____
- Which constraint is more critical for edge deployment? [accuracy/latency]
- How would you prioritize optimization? [accuracy first/latency first/balanced]

### Question 5: Optimization Comparison Analysis
Your compare_optimization_techniques() generates recommendations for different use cases.
Given three optimized models:
- Quantized: 0.8× memory, 2× speed, 0.95× accuracy
- Pruned: 0.3× memory, 1.5× speed, 0.98× accuracy
- Distilled: 0.6× memory, 1.8× speed, 0.92× accuracy

For a mobile app with 50MB model size limit and <100ms latency requirement:
- Which optimization offers best memory reduction? _____
- Which balances all constraints best? _____
- What's the key insight about optimization trade-offs? [no free lunch/specialization wins/measurement guides decisions]

## 🎯 MODULE SUMMARY: Benchmarking

Congratulations! You've built a professional benchmarking system that rivals industry-standard evaluation frameworks!

### Key Accomplishments
- Built comprehensive benchmarking infrastructure with BenchmarkResult, Benchmark, and BenchmarkSuite classes
- Implemented statistical rigor with confidence intervals, variance analysis, and measurement optimization
- Created TinyMLPerf-style standardized benchmarks for reproducible cross-system comparison
- Developed optimization comparison workflows that generate actionable recommendations
- All tests pass ✅ (validated by `test_module()`)

### Systems Engineering Insights Gained
- **Measurement Science**: Statistical significance requires proper sample sizes and variance control
- **Benchmark Design**: Standardized protocols enable fair comparison across different systems
- **Trade-off Analysis**: Pareto frontiers reveal optimization opportunities and constraints
- **Production Integration**: Automated reporting transforms measurements into engineering decisions

### Ready for Systems Capstone
Your benchmarking implementation enables the final milestone: a comprehensive systems evaluation comparing CNN vs TinyGPT with quantization, pruning, and performance analysis. This is where all 19 modules come together!

Export with: `tito module complete 19`

**Next**: Milestone 5 (Systems Capstone) will demonstrate the complete ML systems engineering workflow!